# Narwhals howto


Links útiles:
- https://github.com/narwhals-dev/narwhals
- https://2024.pycon.it/en/event/how-you-can-write-a-dataframe-agnostic-library
- https://labs.quansight.org/blog/dataframe-interop-pycon-lit-2024
- https://labs.quansight.org/blog/scikit-lego-narwhals

## Motivación
- No queremos tener funciones o libs que estén atadas al tipo de dataframe que usamos, no vamos a replicarla y tener una para polars, otra para padas, modin, cuDF, etc.
- Tampoco pretendemos ir detras de cada nueva lib y andar haciendo migraciones masivas de nuestro código
- Ni queremos obligar a que usen un determinado dataframe, si te sentís bien con pandas, seguí usándolo, yo no te voy a molestar

**¿Qué es?**
Es una capa de abstracción que agregamos a nuestras funciones y nos olvidamos del problema. Veamos un ejemplo concreto para una función que da soporte para dataframe pandas y polars:

In [47]:
import polars as pl
import pandas as pd
import narwhals as nw

In [48]:
print(f"Polars: v{pl.__version__} / Pandas: v{pd.__version__} / Narwhals: v{nw.__version__}")

Polars: v1.10.0 / Pandas: v2.2.3 / Narwhals: v1.18.3


In [40]:
def has_artist_bad(artist, df):
    # para pandas
    if isinstance(df, pd.DataFrame):
        return artist in df["artist"]
    # para polars
    if isinstance(df, (pl.DataFrame, pl.LazyFrame)):
        return artist in df["artist"]

In [52]:
data = {"artist":["charly", "divididos", "sumo", "divididos"],
        "cd":["clics modernos", "amapola del 66", "Llegando los monos", "amapola del 66"], 
        "song":["Ojos de video tape", "La flor azul", "Estallando desde el oceano", "hombres en U"]}
df_pd = pd.DataFrame(data)
df_pl = pl.DataFrame(data)

In [53]:
has_artist_bad("charly", df_pd)

False

In [54]:
has_artist_bad("charly", df_pl)

True

- Ajá, acá vemos que se comportan diferentes, resulta que el operador in en Pandas busca en los índices de la serie en vez de su contenido, por eso retorna falso. Vamos a arreglarlo:

In [55]:
def has_artist_fixed(artist, df):
    # para pandas
    if isinstance(df, pd.DataFrame):
        return artist in df["artist"].values
    # para polars
    if isinstance(df, (pl.DataFrame, pl.LazyFrame)):
        return artist in df["artist"]

In [56]:
has_artist_fixed("charly", df_pd)

True

In [57]:
has_artist_fixed("charly", df_pl)

True

Es absolutamente indeseable ir modificando el código dependiendo del dataframe en cuestión, hagamos nuestras funciones agnósticas al tipo de dataframe. Es aquí donde **Nawhals** viene a asistirnos.  

In [58]:
def has_artist(artist, df_any):
    # dataframe-agnostic
    df = nw.from_native(df_any)
    return artist in df["artist"]

In [59]:
has_artist("charly", df_pd)

True

In [60]:
has_artist("charly", df_pl)

True

Hasta acá todo muy lindo pero hagamos una función que me recibe un dataframe, lo procesa y retorna modificado en el formato que el usuario de la función lo envió.
- Recibimos el dataframe y retornemos otro con la cantidad de canciones por artista

In [66]:
def songs_by_artist(df_any):
    df = nw.from_native(df_any)
    result = df.group_by('artist').agg(nw.len())
    return nw.to_native(result)

In [68]:
res_pd = songs_by_artist(df_pd)
res_pl = songs_by_artist(df_pl)

In [70]:
type(res_pd)

pandas.core.frame.DataFrame

In [71]:
res_pd

,artist,len
0,charly,1
1,divididos,2
2,sumo,1


In [72]:
type(res_pl)

polars.dataframe.frame.DataFrame

In [73]:
res_pl

artist,len
str,u32
"""charly""",1
"""sumo""",1
"""divididos""",2


### Tras bambalinas
Narwhals usa un subset de la API de Polars, así que no hay que aprender nada (siempre y cuando ya sepas Polars). Una referencia de lo [soportado acá](https://narwhals-dev.github.io/narwhals/api-reference/). Si bien soporta múltiples dataframes, tiene un coverage del 100% para pandas y polars (testeado diariamente!).

Narwhals tiene una sobrecarga despreciable ya que no hace conversiones entre diferentes tipos de dataframes sino que convierte la sintaxis, tampoco tiene dependencias, usa la tecnología del dataframe que pasa el usuario. [No va a acelerar ni tampoco ralentizar tu proceso](https://narwhals-dev.github.io/narwhals/overhead/) solamente lo va a hacer transparente a diferentes tecnologías de dataframes.

Detrás de Narwhals están el/los [core-developers de Pandas y Polars](https://uk.linkedin.com/in/marcogorelli). Algunos hitos:
- Elegida como una de las [top-10 libs del 2024](https://tryolabs.com/blog/top-python-libraries-2024#top-10---aimldata)
- Plotly la incluyó como una dependencia
- Sklearn-lego la incorporó para dar soporte a múltiples dataframes